# PRÁCTICA B2-2 #

## MÓDULO DE GESTIÓN DE RIESGOS ##
### Escenarios de Estrés y Cambios de Régimen de Mercado ###

### Datos básicos: ###
- Práctica en grupos de dos personas
- Entrega el día 15 de febrero a través del aula virtual.
- Los entregables son un notebook de Python y un resumen ejecutivo en formato PDF.

### Objetivo de la práctica ###
El objetivo de esta práctica es rediseñar un motor de stress testing en Python capaz de
capturar el riesgo de cola y los cambios de régimen, identificar cuándo el mercado entra en
“crisis” y cuantificar el riesgo real cuando la diversificación desaparece. El motor de
simulación deberá utilizarse explícitamente para construir Escenarios de Estrés cuyo
objetivo sea “romper la cartera”, forzando condiciones adversas y económicamente
coherentes, y cuantificando pérdidas extremas mediante VaR del 99% y Expected Shortfall
(CVaR).

### Fase 0 - Preparacion y Estructura del Proyecto ###

### Librerias ###

In [52]:
from dataclasses import dataclass, field
from datetime import date
from pathlib import Path
from typing import Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from hmmlearn import hmm
from pandas_datareader import data as pdr
from sklearn.preprocessing import StandardScaler
from scipy import stats as sp_stats

# For PDF generation
try:
    import markdown
    from weasyprint import HTML, CSS
    PDF_AVAILABLE = True
except ImportError:
    PDF_AVAILABLE = False
    print("Warning: markdown/weasyprint not available. PDF generation will be skipped.")

### Variables ###

In [53]:
RANDOM_SEED = 42

BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
DATA_BRONZE_DIR = DATA_DIR / "bronze"
DATA_SILVER_DIR = DATA_DIR / "silver"
DATA_GOLD_DIR = DATA_DIR / "gold"
FIGURES_DIR = BASE_DIR / "figures"
REPORT_DIR = BASE_DIR / "report"

START_DATE = "2006-01-01"
END_DATE = date.today().isoformat()

COMBINED_PATH = DATA_GOLD_DIR / "market_data_combined.csv"

### Clases ###

In [54]:
@dataclass
class MarketData:
    """Utility class for downloading and combining market data."""

    equities: List[str] = field(default_factory=list)
    yields: List[str] = field(default_factory=list)

    combined_data: pd.DataFrame = field(init=False)

    def __post_init__(self) -> None:
        equity_data = (
            self.fetch_equities(self.equities, start=START_DATE, end=END_DATE)
            if self.equities
            else pd.DataFrame()
        )
        yield_data = (
            self.fetch_us_yields(self.yields, start=START_DATE, end=END_DATE)
            if self.yields
            else pd.DataFrame()
        )
        self.combined_data = self.combine_and_fill(equity_data, yield_data)

    @staticmethod
    def fetch_equities(tickers: List[str], start: str, end: str) -> pd.DataFrame:
        """Fetch adjusted close prices for a list of tickers using yfinance."""
        if not tickers:
            return pd.DataFrame()

        equities = yf.download(
            tickers,
            start=start,
            end=end,
            progress=False,
            threads=True,
            auto_adjust=True,
        )["Close"]

        if isinstance(tickers, list):
            tickers_join = "_".join(tickers)
        else:
            tickers_join = str(tickers)

        DATA_BRONZE_DIR.mkdir(parents=True, exist_ok=True)
        equities_path = DATA_BRONZE_DIR / f"equities_adj_close_{tickers_join}.csv"
        equities.sort_index().to_csv(equities_path)

        return equities.sort_index()

    @staticmethod
    def fetch_us_yields(tickers: Union[List[str], str], start: str, end: str) -> pd.DataFrame:
        """Fetch US yields from FRED."""
        if not tickers:
            return pd.DataFrame()

        yields = pdr.DataReader(tickers, "fred", start, end)
        yields.index = pd.to_datetime(yields.index)

        if isinstance(tickers, list):
            tickers_join = "_".join(tickers)
        else:
            tickers_join = str(tickers)

        DATA_BRONZE_DIR.mkdir(parents=True, exist_ok=True)
        yields_path = DATA_BRONZE_DIR / f"us_yields_{tickers_join}.csv"
        yields.sort_index().to_csv(yields_path)

        return yields.sort_index()

    @staticmethod
    def combine_and_fill(equities: pd.DataFrame, yields: pd.DataFrame) -> pd.DataFrame:
        """Combine equities and yields into a single DataFrame and forward-fill missing yield data."""
        combined = pd.concat([equities, yields], axis=1).sort_index()

        # Forward-fill only yield series to avoid contaminating equity prices
        for col in ["GS10", "GS2", "BAMLH0A0HYM2"]:
            if col in combined.columns:
                combined[col] = combined[col].ffill()

        DATA_SILVER_DIR.mkdir(parents=True, exist_ok=True)
        combined_path = DATA_SILVER_DIR / "market_data_combined.csv"
        combined.to_csv(combined_path)

        return combined

@dataclass
class Portfolio:
    """Equal-weight portfolio built from equities and yield instruments."""

    assets: Dict[str, str]

    prices: pd.DataFrame = field(init=False)
    returns: pd.DataFrame = field(init=False)
    weights: pd.DataFrame = field(init=False)
    portfolio_returns: pd.Series = field(init=False)

    def __post_init__(self) -> None:
        self._load_prices()
        self._compute_returns()
        self._compute_dynamic_weights()
        self._compute_portfolio_returns()

    def _load_prices(self) -> None:
        equities = [
            ticker for ticker, asset_type in self.assets.items() if asset_type == "equity"
        ]
        yields = [
            ticker for ticker, asset_type in self.assets.items() if asset_type == "yield"
        ]

        equity_data = MarketData.fetch_equities(equities, start=START_DATE, end=END_DATE)
        yield_data = MarketData.fetch_us_yields(yields, start=START_DATE, end=END_DATE)

        self.prices = MarketData.combine_and_fill(equity_data, yield_data)

    def _compute_returns(self) -> None:
        # Daily simple returns without implicit forward-filling
        self.returns = self.prices.pct_change(fill_method=None)

    def _compute_dynamic_weights(self) -> None:
        asset_exists = ~self.prices.isna()
        n_assets = asset_exists.sum(axis=1)

        self.weights = asset_exists.div(n_assets, axis=0).fillna(0.0)

    def _compute_portfolio_returns(self) -> None:
        self.portfolio_returns = (self.returns * self.weights).sum(axis=1)

    def cumulative_return(self) -> pd.Series:
        return (1 + self.portfolio_returns).cumprod()

    def drawdown(self) -> pd.Series:
        wealth = self.cumulative_return()
        peak = wealth.cummax()
        return (wealth - peak) / peak

    def max_drawdown(self) -> float:
        return float(self.drawdown().min())

    def volatility(self, annualized: bool = True) -> float:
        vol = float(self.portfolio_returns.std())
        return vol * np.sqrt(252) if annualized else vol

    def mean_return(self, annualized: bool = True) -> float:
        mu = float(self.portfolio_returns.mean())
        return mu * 252 if annualized else mu

    def sharpe_ratio(self) -> float:
        return self.mean_return() / self.volatility()

    def var_cvar(self, alpha: float = 0.99) -> Tuple[float, float]:
        var = float(self.portfolio_returns.quantile(1 - alpha))
        cvar = float(self.portfolio_returns[self.portfolio_returns <= var].mean())
        return var, cvar

    def summary(self) -> pd.Series:
        var_99, cvar_99 = self.var_cvar(0.99)

        return pd.Series(
            {
                "Mean Return (ann)": self.mean_return(),
                "Volatility (ann)": self.volatility(),
                "Sharpe": self.sharpe_ratio(),
                "Max Drawdown": self.max_drawdown(),
                "VaR 99%": var_99,
                "CVaR 99%": cvar_99,
            }
        )

    def portfolio_composition_table(self) -> pd.DataFrame:
        weights_pct = self.weights * 100
        asset_values = self.prices * self.weights

        data: Dict[Tuple[str, str], pd.Series] = {}
        for asset in self.prices.columns:
            data[(asset, "weight_%")] = weights_pct[asset]
            data[(asset, "price")] = self.prices[asset]
            data[(asset, "value")] = asset_values[asset]

        df = pd.DataFrame(data)
        df.columns = pd.MultiIndex.from_tuples(df.columns)

        df["portfolio_value"] = asset_values.sum(axis=1)
        df["portfolio_return_%"] = df["portfolio_value"].pct_change() * 100

        DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
        portfolio_table_path = DATA_GOLD_DIR / "portfolio_composition.csv"
        df.to_csv(portfolio_table_path)

        return df

    def plot_portfolio(self) -> None:
        cumulative_returns = self.cumulative_return()
        plt.figure(figsize=(10, 3))
        sns.lineplot(data=cumulative_returns)
        plt.title("Cumulative Return of the Portfolio")
        plt.xlabel("Date")
        plt.ylabel("Cumulative Return")
        plt.tight_layout()
        FIGURES_DIR.mkdir(parents=True, exist_ok=True)
        chart_path = FIGURES_DIR / "portfolio_returns_chart.png"
        plt.savefig(chart_path)
        plt.close()

    def plot_chart_per_asset(self) -> None:
        FIGURES_DIR.mkdir(parents=True, exist_ok=True)
        for col in self.prices.columns:
            plt.figure(figsize=(10, 3))
            sns.lineplot(data=self.prices[col])
            plt.title(f"{col} Price Over Time")
            plt.xlabel("Date")
            plt.ylabel("Price")
            plt.tight_layout()
            chart_path = FIGURES_DIR / f"{col}_price_chart.png"
            plt.savefig(chart_path)
            plt.close()

@dataclass
class HMMState:
    """Parameters of a single HMM state."""

    mean: np.ndarray
    cov: np.ndarray
    volatility: float  # Frobenius norm of covariance diagonal

@dataclass
class HMMResults:
    """Fitted HMM results and regime assignments."""

    model: hmm.GaussianHMM
    transition_matrix: np.ndarray
    states: Dict[int, HMMState]
    regimes: np.ndarray  # regime_t for each time step
    calm_state: int  # which state index corresponds to "calm"
    crisis_state: int  # which state index corresponds to "crisis"

@dataclass
class RegimeCopula:
    """Copula parameters for a given regime (simple Gaussian copula)."""

    regime_name: str
    assets: List[str]
    correlation: pd.DataFrame  # correlation matrix in asset order

    def sample(self, n_samples: int, random_state: Union[int, None] = None) -> np.ndarray:
        """Draw samples from the copula in standard normal space."""
        rng = np.random.default_rng(random_state)
        cov = self.correlation.values
        chol = np.linalg.cholesky(cov)
        z = rng.standard_normal(size=(n_samples, len(self.assets)))
        return z @ chol.T

@dataclass
class RegimeMonteCarloSimulator:
    """Monte Carlo engine driven by HMM regimes and copulas."""

    transition_matrix: np.ndarray  # from HMM
    assets: List[str]
    regime_marginals: Dict[str, pd.DataFrame]  # output of calculate_marginal_statistics
    regime_copulas: Dict[str, RegimeCopula]

    def _simulate_regime_paths(
        self,
        n_paths: int,
        n_steps: int,
        initial_state: int,
    ) -> np.ndarray:
        """Simulate Markov regime paths using the HMM transition matrix."""
        rng = np.random.default_rng(RANDOM_SEED)
        regimes = np.empty((n_paths, n_steps), dtype=int)
        regimes[:, 0] = initial_state

        for t in range(1, n_steps):
            for p in range(n_paths):
                current = regimes[p, t - 1]
                probs = self.transition_matrix[current]
                regimes[p, t] = rng.choice(len(probs), p=probs)

        return regimes

    def _get_marginals_for_regime(self, regime_label: str) -> Tuple[np.ndarray, np.ndarray]:
        """Return mean and volatility vectors aligned with self.assets for a regime."""
        stats_df = self.regime_marginals[regime_label]
        means = np.array(
            [stats_df.loc[stats_df["Asset"] == a, "Mean Return"].iloc[0] for a in self.assets]
        )
        vols = np.array(
            [stats_df.loc[stats_df["Asset"] == a, "Volatility"].iloc[0] for a in self.assets]
        )
        return means, vols

    def simulate_returns(
        self,
        n_paths: int,
        n_steps: int,
        initial_state: int,
    ) -> Tuple[np.ndarray, np.ndarray]:
        """Simulate multi-asset returns and corresponding regime paths."""
        regimes = self._simulate_regime_paths(n_paths, n_steps, initial_state)
        n_assets = len(self.assets)
        simulated = np.zeros((n_paths, n_steps, n_assets), dtype=float)

        state_to_label = {0: "CALM", 1: "CRISIS"}
        rng = np.random.default_rng(RANDOM_SEED + 1)

        for p in range(n_paths):
            for t in range(n_steps):
                numeric_state = regimes[p, t]
                regime_label = state_to_label[numeric_state]

                if regime_label not in self.regime_copulas:
                    continue

                copula = self.regime_copulas[regime_label]
                means, vols = self._get_marginals_for_regime(regime_label)

                z = copula.sample(1, random_state=rng.integers(0, 1_000_000))[0]
                simulated[p, t, :] = means + vols * z

        return simulated, regimes

@dataclass
class StressScenario:
    """Configuration for a stress-testing scenario in Phase 5."""

    name: str
    description: str
    transition_matrix: np.ndarray
    volatility_multipliers: Dict[str, float]


### Funciones ###

In [ ]:
def set_global_seed() -> None:
    """Set global random seed for reproducibility."""
    np.random.seed(RANDOM_SEED)

def ensure_directories() -> None:
    """Create all necessary directories for the project."""
    DATA_BRONZE_DIR.mkdir(parents=True, exist_ok=True)
    DATA_SILVER_DIR.mkdir(parents=True, exist_ok=True)
    DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
    FIGURES_DIR.mkdir(parents=True, exist_ok=True)
    REPORT_DIR.mkdir(parents=True, exist_ok=True)

# =============================================================================
# FASE 0 - GET DATA
# =============================================================================
def yield_curve_slope(y10: pd.Series, y2: pd.Series) -> pd.Series:
    """Calculate the yield curve slope (10Y - 2Y spread)."""
    return y10 - y2

def portfolio() -> Portfolio:
    """Create the baseline multi-asset portfolio configuration and return a Portfolio."""

    assets: Dict[str, str] = {
        "AAPL": "equity",
        "AMZN": "equity",
        "BAC": "equity",
        "BRK-B": "equity",
        "CVX": "equity",
        "ENPH": "equity",
        "GLD": "equity",
        "GME": "equity",
        "GOOGL": "equity",
        "JNJ": "equity",
        "JPM": "equity",
        "MSFT": "equity",
        "NVDA": "equity",
        "PG": "equity",
        "XOM": "equity",
        "HYG": "equity",
        "GS10": "yield",
        "GS2": "yield",
    }

    return Portfolio(assets=assets)

def market_risk() -> pd.DataFrame:
    """Construct the market risk data set used for regime detection."""

    # Equity market
    sp500 = MarketData.fetch_equities(tickers=["^GSPC"], start=START_DATE, end=END_DATE)
    sp500_ret = sp500.pct_change(fill_method=None)

    vix = MarketData.fetch_equities(tickers=["^VIX"], start=START_DATE, end=END_DATE)
    vix_ret = vix.pct_change(fill_method=None)

    # Interest rates
    y10 = MarketData.fetch_us_yields("GS10", start=START_DATE, end=END_DATE)
    y2 = MarketData.fetch_us_yields("GS2", start=START_DATE, end=END_DATE)

    y10_chg = y10.pct_change(fill_method=None)
    y2_chg = y2.pct_change(fill_method=None)
    slope = yield_curve_slope(y10["GS10"], y2["GS2"]).rename("yield_slope")

    # Credit spread
    hy_spread = MarketData.fetch_us_yields("BAMLH0A0HYM2", start=START_DATE, end=END_DATE)
    hy_spread_chg = hy_spread.pct_change(fill_method=None)

    # Combine all market risk drivers
    df = pd.concat(
        [
            sp500_ret,
            vix_ret,
            y10_chg,
            y2_chg,
            slope,
            hy_spread_chg,
        ],
        axis=1,
    )

    for col in ["GS10", "GS2", "yield_slope"]:
        if col in df.columns:
            df[col] = df[col].ffill()

    DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
    df_path = DATA_GOLD_DIR / "market_data_combined.csv"
    df.to_csv(df_path)

    return df

# =============================================================================
# FASE 1 - Detectando el "Pulso" del Mercado (Hidden Markov Models)
# =============================================================================#Fase 1
def separate_data_by_regime(
    portfolio: Portfolio,
    regimes: np.ndarray,
    log_returns_clean: pd.DataFrame,
    hmm_results: HMMResults
) -> Tuple[Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]:
    """Separate portfolio returns and asset prices by regime.
    
    Parameters
    ----------
    portfolio : Portfolio
        Portfolio object with prices and returns.
    regimes : np.ndarray
        Regime assignments.
    log_returns_clean : pd.DataFrame
        Cleaned log returns aligned with regimes.
    hmm_results : HMMResults
        HMM results with state labels.
    
    Returns
    -------
    Tuple[Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]
        Returns and prices separated by regime.
    """
    # Align portfolio returns with cleaned log returns index
    portfolio_returns_clean = portfolio.portfolio_returns.loc[log_returns_clean.index]
    
    # Separate data by regime
    calm_mask = regimes == hmm_results.calm_state
    crisis_mask = regimes == hmm_results.crisis_state
    
    returns_by_regime = {
        "CALM": portfolio_returns_clean[calm_mask],
        "CRISIS": portfolio_returns_clean[crisis_mask]
    }
    
    # Separate asset returns by regime
    asset_returns_by_regime = {
        "CALM": portfolio.returns.loc[log_returns_clean.index][calm_mask],
        "CRISIS": portfolio.returns.loc[log_returns_clean.index][crisis_mask]
    }
    
    return returns_by_regime, asset_returns_by_regime

def calculate_marginal_statistics(
    asset_returns: Dict[str, pd.DataFrame],
    assets: Dict[str, str]
) -> pd.DataFrame:
    """Calculate marginal statistics (mean, vol, skewness, kurtosis) by regime and asset.
    
    Parameters
    ----------
    asset_returns : Dict[str, pd.DataFrame]
        Asset returns separated by regime.
    assets : Dict[str, str]
        Asset dictionary with types.
    
    Returns
    -------
    pd.DataFrame
        Comprehensive statistics table by asset and regime.
    """
    stats_list = []
    
    for regime_name, returns_df in asset_returns.items():
        for asset in returns_df.columns:
            if asset in assets:
                asset_ret = returns_df[asset].dropna()
                
                if len(asset_ret) > 0:
                    mean_ret = asset_ret.mean()
                    volatility = asset_ret.std()
                    skewness = sp_stats.skew(asset_ret)
                    kurtosis = sp_stats.kurtosis(asset_ret)
                    
                    stats_list.append({
                        "Asset": asset,
                        "Regime": regime_name,
                        "Mean Return": mean_ret,
                        "Volatility": volatility,
                        "Skewness": skewness,
                        "Kurtosis": kurtosis,
                        "N Obs": len(asset_ret)
                    })
    
    df_stats = pd.DataFrame(stats_list)
    return df_stats.sort_values(["Asset", "Regime"]).reset_index(drop=True)

def analyze_key_assets(
    asset_returns: Dict[str, pd.DataFrame],
    key_assets: List[str] = ["HYG", "GLD"]
) -> pd.DataFrame:
    """Focus analysis on key assets (High Yield, Gold).
    
    Parameters
    ----------
    asset_returns : Dict[str, pd.DataFrame]
        Asset returns separated by regime.
    key_assets : List[str]
        List of key assets to analyze.
    
    Returns
    -------
    pd.DataFrame
        Detailed statistics for key assets.
    """
    key_stats = []
    
    for asset in key_assets:
        for regime_name, returns_df in asset_returns.items():
            if asset in returns_df.columns:
                asset_ret = returns_df[asset].dropna()
                
                if len(asset_ret) > 0:
                    var_99 = asset_ret.quantile(0.01)  # 1% worst case
                    cvar_99 = asset_ret[asset_ret <= var_99].mean()
                    
                    key_stats.append({
                        "Asset": asset,
                        "Regime": regime_name,
                        "Mean (%)": asset_ret.mean() * 100,
                        "Volatility (%)": asset_ret.std() * 100,
                        "Skewness": sp_stats.skew(asset_ret),
                        "Kurtosis": sp_stats.kurtosis(asset_ret),
                        "VaR 99%": var_99,
                        "CVaR 99%": cvar_99,
                        "Min Return": asset_ret.min(),
                        "Max Return": asset_ret.max(),
                    })
    
    df_key = pd.DataFrame(key_stats)
    return df_key.sort_values(["Asset", "Regime"]).reset_index(drop=True)

def interpret_regime_changes(df_key_assets: pd.DataFrame) -> str:
    """Generate economic interpretation of regime changes for key assets.
    
    Parameters
    ----------
    df_key_assets : pd.DataFrame
        Key assets statistics by regime.
    
    Returns
    -------
    str
        Interpretation text.
    """
    interpretation = []
    interpretation.append("=" * 80)
    interpretation.append("INTERPRETACIÓN ECONÓMICA DE CAMBIOS DE RÉGIMEN")
    interpretation.append("=" * 80 + "\n")
    
    # HYG Analysis
    hyg_calm = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CALM")]
    hyg_crisis = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CRISIS")]
    
    if not hyg_calm.empty and not hyg_crisis.empty:
        vol_calm = hyg_calm["Volatility (%)"].values[0]
        vol_crisis = hyg_crisis["Volatility (%)"].values[0]
        vol_change = ((vol_crisis - vol_calm) / vol_calm) * 100
        
        interpretation.append("📊 HIGH YIELD (HYG) - Bonos de Alto Rendimiento")
        interpretation.append("-" * 80)
        interpretation.append(f"  • Volatilidad en CALMA: {vol_calm:.2f}%")
        interpretation.append(f"  • Volatilidad en CRISIS: {vol_crisis:.2f}%")
        interpretation.append(f"  • Aumento: {vol_change:.1f}%")
        interpretation.append("\n  INTERPRETACIÓN:")
        interpretation.append("  El aumento de volatilidad en crisis refleja:")
        interpretation.append("  ✓ Mayor aversión al riesgo en el mercado")
        interpretation.append("  ✓ Widening de spreads de crédito")
        interpretation.append("  ✓ Stress en el segmento de bonos de alto rendimiento")
        interpretation.append("  → El HYG es PRO-CÍCLICO (amplifica riesgo en crisis)\n")
    
    # GLD Analysis
    gld_calm = df_key_assets[(df_key_assets["Asset"] == "GLD") & (df_key_assets["Regime"] == "CALM")]
    gld_crisis = df_key_assets[(df_key_assets["Asset"] == "GLD") & (df_key_assets["Regime"] == "CRISIS")]
    
    if not gld_calm.empty and not gld_crisis.empty:
        ret_calm = gld_calm["Mean (%)"].values[0]
        ret_crisis = gld_crisis["Mean (%)"].values[0]
        vol_calm_gld = gld_calm["Volatility (%)"].values[0]
        vol_crisis_gld = gld_crisis["Volatility (%)"].values[0]
        
        interpretation.append("🏆 ORO (GLD) - Activo Refugio")
        interpretation.append("-" * 80)
        interpretation.append(f"  • Retorno medio en CALMA: {ret_calm:.2f}%")
        interpretation.append(f"  • Retorno medio en CRISIS: {ret_crisis:.2f}%")
        interpretation.append(f"  • Volatilidad en CALMA: {vol_calm_gld:.2f}%")
        interpretation.append(f"  • Volatilidad en CRISIS: {vol_crisis_gld:.2f}%")
        
        if ret_crisis > ret_calm:
            interpretation.append("\n  INTERPRETACIÓN:")
            interpretation.append("  ✓ El ORO SUBE durante crisis (comportamiento de refugio)")
            interpretation.append("  ✓ Inversores huyen a activos seguros")
            interpretation.append("  ✓ Cobertura contra inflación y depreciación de divisas")
            interpretation.append("  → El GLD es ANTI-CÍCLICO (protección en turbulencia)\n")
        else:
            interpretation.append("\n  INTERPRETACIÓN:")
            interpretation.append("  ⚠ El ORO NO actúa como refugio esperado")
            interpretation.append("  ⚠ Posible liquidación forzada en crisis")
            interpretation.append("  → Revisar correlación con equity en stress\n")
    
    interpretation.append("=" * 80)
    return "\n".join(interpretation)

def compare_volatility_regimes(df_stats: pd.DataFrame) -> pd.DataFrame:
    """Create comparison table of volatility changes between regimes.
    
    Parameters
    ----------
    df_stats : pd.DataFrame
        Statistics by asset and regime.
    
    Returns
    -------
    pd.DataFrame
        Volatility comparison table.
    """
    vol_comparison = []
    
    for asset in df_stats["Asset"].unique():
        asset_data = df_stats[df_stats["Asset"] == asset]
        
        calm_vol = asset_data[asset_data["Regime"] == "CALM"]["Volatility"].values
        crisis_vol = asset_data[asset_data["Regime"] == "CRISIS"]["Volatility"].values
        
        if len(calm_vol) > 0 and len(crisis_vol) > 0:
            vol_ratio = crisis_vol[0] / calm_vol[0]
            vol_change = ((crisis_vol[0] - calm_vol[0]) / calm_vol[0]) * 100
            
            vol_comparison.append({
                "Asset": asset,
                "Volatility CALM": calm_vol[0],
                "Volatility CRISIS": crisis_vol[0],
                "Ratio (Crisis/Calm)": vol_ratio,
                "% Change": vol_change
            })
    
    df_vol = pd.DataFrame(vol_comparison)
    return df_vol.sort_values("Ratio (Crisis/Calm)", ascending=False).reset_index(drop=True)

def save_phase1_analysis(
    df_stats: pd.DataFrame,
    df_key_assets: pd.DataFrame,
    df_vol_comparison: pd.DataFrame,
    interpretation: str,
    output_dir: Path
) -> None:
    """Save all Phase 1 analysis results to files.
    
    Parameters
    ----------
    df_stats : pd.DataFrame
        Marginal statistics table.
    df_key_assets : pd.DataFrame
        Key assets analysis.
    df_vol_comparison : pd.DataFrame
        Volatility comparison.
    interpretation : str
        Economic interpretation text.
    output_dir : Path
        Output directory path.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save statistics tables
    df_stats.to_csv(output_dir / "phase1_marginal_statistics.csv", index=False)
    df_key_assets.to_csv(output_dir / "phase1_key_assets_analysis.csv", index=False)
    df_vol_comparison.to_csv(output_dir / "phase1_volatility_comparison.csv", index=False)
    
    # Save interpretation with UTF-8 encoding
    with open(output_dir / "phase1_interpretation.txt", "w", encoding="utf-8") as f:
        f.write(interpretation)

def run_phase1_risk_analysis(
    portfolio: Portfolio,
    regimes: np.ndarray,
    log_returns_clean: pd.DataFrame,
    hmm_results: HMMResults
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, str]:
    """Execute complete Phase 1 analysis: Risk by Regime.
    
    Parameters
    ----------
    portfolio : Portfolio
        Portfolio object with prices and returns.
    regimes : np.ndarray
        Regime assignments.
    log_returns_clean : pd.DataFrame
        Cleaned log returns aligned with regimes.
    hmm_results : HMMResults
        HMM results with state labels.
    
    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, str]
        Marginal statistics, key assets analysis, volatility comparison, and interpretation.
    """
    
    print("=" * 80)
    print("FASE 1: ANÁLISIS DE RIESGO INDIVIDUAL POR RÉGIMEN")
    print("=" * 80 + "\n")
    
    # Task 1.1: Separate data by regime
    print("1.1 Separando datos por régimen...")
    returns_by_regime, asset_returns_by_regime = separate_data_by_regime(
        portfolio, regimes, log_returns_clean, hmm_results
    )
    
    calm_obs = len(returns_by_regime["CALM"])
    crisis_obs = len(returns_by_regime["CRISIS"])
    print(f"     ✓ Días en CALMA: {calm_obs}")
    print(f"     ✓ Días en CRISIS: {crisis_obs}\n")
    
    # Task 1.2: Calculate marginal statistics
    print("1.2 Calculando estadísticas marginales...")
    df_stats = calculate_marginal_statistics(asset_returns_by_regime, portfolio.assets)
    print(f"     ✓ {len(df_stats)} filas de estadísticas (activos × regímenes)\n")
    
    # Task 1.3: Analyze key assets
    print("1.3 Analizando activos clave (HYG, GLD)...")
    df_key_assets = analyze_key_assets(asset_returns_by_regime)
    print(f"     ✓ Análisis detallado de {df_key_assets['Asset'].nunique()} activos\n")
    
    # Volatility comparison
    print("1.4 Comparando volatilidades entre regímenes...")
    df_vol_comparison = compare_volatility_regimes(df_stats)
    print(f"     ✓ Tabla de comparación creada\n")
    
    # Task 1.4: Economic interpretation
    print("1.4 Generando interpretación económica...")
    interpretation = interpret_regime_changes(df_key_assets)
    print(interpretation)
    print()
    
    # Save results
    print("Guardando resultados de Fase 1...")
    save_phase1_analysis(df_stats, df_key_assets, df_vol_comparison, interpretation, DATA_GOLD_DIR)
    print(f"     ✓ Resultados guardados en {DATA_GOLD_DIR}\n")
    
    return df_stats, df_key_assets, df_vol_comparison, interpretation

# =============================================================================
# FASE 2 -  Anatomía del Riesgo (Análisis Marginal)
# =============================================================================
def load_and_prepare_returns(data_path: Path) -> Tuple[pd.DataFrame, pd.Series]:
    """Load combined market data and prepare log returns for HMM.
    
    Parameters
    ----------
    data_path : Path
        Path to the combined market data CSV file.
    
    Returns
    -------
    Tuple[pd.DataFrame, pd.Series]
        Log returns DataFrame and S&P 500 price series.
    """
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
    
    # Calculate log returns from the returns data
    log_returns = df.copy()
    
    # Load original S&P 500 prices from Bronze directory for visualization
    sp500_bronze_path = DATA_BRONZE_DIR / "equities_adj_close_^GSPC.csv"
    if sp500_bronze_path.exists():
        sp500_prices_raw = pd.read_csv(sp500_bronze_path, index_col=0, parse_dates=True)
        # The column name should be the ticker itself
        sp500_prices = sp500_prices_raw.iloc[:, 0]  # Get first column regardless of name
    else:
        # Fallback: reconstruct from returns if Bronze file not available
        sp500_prices = pd.Series(index=log_returns.index, dtype=float)
    
    return log_returns, sp500_prices

def standardize_returns(log_returns: pd.DataFrame) -> Tuple[np.ndarray, StandardScaler, pd.DataFrame]:
    """Standardize log returns using StandardScaler.
    
    Parameters
    ----------
    log_returns : pd.DataFrame
        DataFrame with log returns.
    
    Returns
    -------
    Tuple[np.ndarray, StandardScaler, pd.DataFrame]
        Scaled returns array, fitted scaler object, and cleaned returns DataFrame.
    """
    # Remove rows with NaN or infinity values
    log_returns_clean = log_returns.replace([np.inf, -np.inf], np.nan).dropna()
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(log_returns_clean)
    return X_scaled, scaler, log_returns_clean

def save_hmm_parameters(hmm_results: HMMResults, output_path: Path) -> None:
    """Save HMM parameters to text file.
    
    Parameters
    ----------
    hmm_results : HMMResults
        HMM results container.
    output_path : Path
        Path to save parameters file.
    """
    with open(output_path, "w") as f:
        f.write("=" * 80 + "\n")
        f.write("HMM PARAMETERS\n")
        f.write("=" * 80 + "\n\n")
        
        f.write("TRANSITION MATRIX:\n")
        f.write(str(hmm_results.transition_matrix) + "\n\n")
        
        for state_idx, state in hmm_results.states.items():
            state_label = "CALM" if state_idx == hmm_results.calm_state else "CRISIS"
            f.write(f"\nSTATE {state_idx} ({state_label}):\n")
            f.write(f"  Volatility: {state.volatility:.6f}\n")
            f.write(f"  Mean:\n{state.mean}\n")
            f.write(f"  Covariance (diagonal):\n{np.diag(state.cov)}\n")

def analyze_hmm_features(log_returns: pd.DataFrame, hmm_results: HMMResults) -> pd.DataFrame:
    """Analyze the contribution of each market variable to regime detection.
    
    Shows the mean returns and volatility per feature in each HMM state
    (Calm vs Crisis), demonstrating that all market variables are being used.
    
    Parameters
    ----------
    log_returns : pd.DataFrame
        The multivariate log-returns DataFrame (all market variables).
    hmm_results : HMMResults
        Fitted HMM results containing state parameters.
    
    Returns
    -------
    pd.DataFrame
        Analysis table showing mean and std for each feature in each state.
    """
    
    analysis_data = []
    
    for state_idx, state in hmm_results.states.items():
        state_name = "CALM" if state_idx == hmm_results.calm_state else "CRISIS"
        
        for col_idx, col_name in enumerate(log_returns.columns):
            analysis_data.append({
                "Variable": col_name,
                "Regime": state_name,
                "Mean (HMM)": state.mean[col_idx],
                "Std Dev (HMM)": np.sqrt(state.cov[col_idx, col_idx]),
            })
    
    df_analysis = pd.DataFrame(analysis_data)
    return df_analysis.sort_values(["Variable", "Regime"]).reset_index(drop=True)

def identify_regimes(model: hmm.GaussianHMM, X_scaled: np.ndarray) -> Tuple[np.ndarray, HMMResults]:
    """Identify market regimes using fitted HMM.
    
    Determines which state is "calm" and which is "crisis" based on volatility levels.
    
    Parameters
    ----------
    model : hmm.GaussianHMM
        Fitted HMM model.
    X_scaled : np.ndarray
        Scaled multivariate returns.
    
    Returns
    -------
    Tuple[np.ndarray, HMMResults]
        Regime assignments and full HMM results container.
    """
    regimes = model.predict(X_scaled)
    
    # Determine which state is calm vs crisis based on volatility
    state_volatilities = []
    for i in range(model.n_components):
        vol = np.sqrt(np.trace(model.covars_[i]) / model.n_features)
        state_volatilities.append(vol)
    
    calm_state = int(np.argmin(state_volatilities))
    crisis_state = 1 - calm_state
    
    # Build state parameters
    states = {}
    for i in range(model.n_components):
        states[i] = HMMState(
            mean=model.means_[i],
            cov=model.covars_[i],
            volatility=state_volatilities[i]
        )
    
    hmm_results = HMMResults(
        model=model,
        transition_matrix=model.transmat_,
        states=states,
        regimes=regimes,
        calm_state=calm_state,
        crisis_state=crisis_state
    )
    
    return regimes, hmm_results

def visualize_regimes(
    prices: pd.Series,
    regimes: np.ndarray,
    calm_state: int,
    crisis_state: int,
    plot_path: Path
) -> None:
    """Visualize price series with regime coloring.
    
    Parameters
    ----------
    prices : pd.Series
        Price series to plot.
    regimes : np.ndarray
        Regime assignments.
    calm_state : int
        Index of calm regime.
    crisis_state : int
        Index of crisis regime.
    plot_path : Path
        Path to save the figure.
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    
    # Plot prices
    ax.plot(prices.index, prices.values, "k-", linewidth=1.5, label="S&P 500 Price")
    
    # Color background by regime
    for i in range(len(regimes) - 1):
        if regimes[i] == calm_state:
            ax.axvspan(prices.index[i], prices.index[i + 1], alpha=0.2, color="whitesmoke")
        else:
            ax.axvspan(prices.index[i], prices.index[i + 1], alpha=0.2, color="deepskyblue")
    
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.set_title("Market Regimes: White=Calm, Blue=Crisis")
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.close()

def compute_regime_statistics(regimes: np.ndarray, calm_state: int) -> Dict[str, float]:
    """Compute summary statistics of regime frequencies."""

    crisis_state = 1 - calm_state
    n_calm = int((regimes == calm_state).sum())
    n_crisis = int((regimes == crisis_state).sum())
    pct_calm = 100.0 * n_calm / len(regimes)
    pct_crisis = 100.0 * n_crisis / len(regimes)

    return {
        "n_calm_days": n_calm,
        "n_crisis_days": n_crisis,
        "pct_calm": pct_calm,
        "pct_crisis": pct_crisis,
    }

def fit_hmm(X_scaled: np.ndarray, n_components: int = 2) -> hmm.GaussianHMM:
    """Fit a Gaussian HMM to the scaled returns.
    
    Parameters
    ----------
    X_scaled : np.ndarray
        Scaled multivariate returns.
    n_components : int
        Number of hidden states (default: 2 for calm/crisis).
    
    Returns
    -------
    hmm.GaussianHMM
        Fitted HMM model.
    """
    model = hmm.GaussianHMM(n_components=n_components, random_state=RANDOM_SEED, n_iter=5000)
    model.fit(X_scaled)
    return model

def run_regime_detection_pipeline() -> Tuple[Dict[str, float], np.ndarray, HMMResults, pd.DataFrame, pd.Series]:
    """Run the full HMM-based regime detection workflow and return all key outputs.

    Returns
    -------
    Tuple[Dict[str, float], np.ndarray, HMMResults, pd.DataFrame, pd.Series]
        Regime statistics, regime assignments, HMM results, cleaned log returns,
        and aligned S&P 500 price series.
    """

    # Data preparation
    log_returns, sp500_prices = load_and_prepare_returns(COMBINED_PATH)

    # Display which variables are being analyzed
    print("=" * 80)
    print("MARKET VARIABLES USED FOR REGIME DETECTION (Multivariate Gaussian HMM):")
    print("=" * 80)
    for i, col in enumerate(log_returns.columns, 1):
        print(f"{i}. {col}")
    print(f"\nTotal dimensions: {log_returns.shape[1]} variables × {log_returns.shape[0]} observations")
    print("=" * 80 + "\n")

    X_scaled, _, log_returns_clean = standardize_returns(log_returns)

    # HMM fitting
    model = fit_hmm(X_scaled, n_components=2)

    # Regime identification
    regimes, hmm_results = identify_regimes(model, X_scaled)

    # Analyze feature contributions to regimes (use cleaned returns)
    df_feature_analysis = analyze_hmm_features(log_returns_clean, hmm_results)
    print("FEATURE ANALYSIS - Mean and Volatility per Regime:")
    print("=" * 80)
    print(df_feature_analysis.to_string(index=False))
    print("=" * 80 + "\n")

    # Align sp500_prices with cleaned returns
    sp500_prices_clean = sp500_prices.loc[log_returns_clean.index]

    # Visualization
    FIGURES_DIR.mkdir(parents=True, exist_ok=True)
    plot_path = FIGURES_DIR / "regime_visualization_sp500.png"
    visualize_regimes(
        sp500_prices_clean,
        regimes,
        hmm_results.calm_state,
        hmm_results.crisis_state,
        plot_path,
    )

    # Save outputs (Gold layer)
    DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
    regime_ts_path = DATA_GOLD_DIR / "regime_timeseries.csv"
    save_regime_timeseries(log_returns_clean.index, regimes, sp500_prices_clean, hmm_results, regime_ts_path)

    hmm_params_path = DATA_GOLD_DIR / "hmm_parameters.txt"
    save_hmm_parameters(hmm_results, hmm_params_path)

    # Statistics
    stats = compute_regime_statistics(regimes, hmm_results.calm_state)
    return stats, regimes, hmm_results, log_returns_clean, sp500_prices_clean

def save_regime_timeseries(
    dates: pd.DatetimeIndex,
    regimes: np.ndarray,
    sp500_prices: pd.Series,
    hmm_results: HMMResults,
    output_path: Path
) -> None:
    """Save regime time series to CSV.
    
    Parameters
    ----------
    dates : pd.DatetimeIndex
        Index dates.
    regimes : np.ndarray
        Regime assignments.
    sp500_prices : pd.Series
        S&P 500 prices.
    hmm_results : HMMResults
        HMM results container.
    output_path : Path
        Path to save CSV.
    """
    df_regimes = pd.DataFrame({
        "date": dates,
        "regime": regimes,
        "regime_label": ["CALM" if r == hmm_results.calm_state else "CRISIS" for r in regimes],
        "sp500_price": sp500_prices.values
    })
    df_regimes.set_index("date", inplace=True)
    df_regimes.to_csv(output_path)

# =============================================================================
# FASE 3 - Cuando la Diversificación Falla (Cópulas)
# =============================================================================

def compute_correlation_by_regime(
    asset_returns_by_regime: Dict[str, pd.DataFrame]
) -> Dict[str, pd.DataFrame]:
    """Compute Pearson correlation matrices for each regime (CALM/CRISIS)."""
    correlation_matrices: Dict[str, pd.DataFrame] = {}
    for regime_name, returns_df in asset_returns_by_regime.items():
        correlation_matrices[regime_name] = returns_df.dropna(axis=1, how="all").corr()
    return correlation_matrices

def calibrate_gaussian_copulas(
    asset_returns_by_regime: Dict[str, pd.DataFrame]
) -> Dict[str, RegimeCopula]:
    """Calibrate Gaussian copulas for each regime using pseudo-observations."""
    copulas: Dict[str, RegimeCopula] = {}

    for regime_name, returns_df in asset_returns_by_regime.items():
        clean = returns_df.dropna()
        if clean.empty:
            continue

        assets = list(clean.columns)

        # Pseudo-observations U in (0,1) via ranks
        ranks = clean.rank(axis=0, method="average")
        u = (ranks - 0.5) / len(clean)

        # Map to standard normal via inverse CDF
        z = sp_stats.norm.ppf(u)
        corr = pd.DataFrame(np.corrcoef(z.T), index=assets, columns=assets)

        copulas[regime_name] = RegimeCopula(
            regime_name=regime_name,
            assets=assets,
            correlation=corr,
        )

    return copulas

def plot_correlation_heatmaps(
    correlation_matrices: Dict[str, pd.DataFrame],
    output_dir: Path = FIGURES_DIR,
) -> None:
    """Save correlation heatmaps for each regime to the figures directory."""
    output_dir.mkdir(parents=True, exist_ok=True)

    for regime_name, corr in correlation_matrices.items():
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr, cmap="coolwarm", center=0, annot=False)
        plt.title(f"Correlation Matrix - {regime_name} Regime")
        plt.tight_layout()
        fig_path = output_dir / f"correlation_matrix_{regime_name.lower()}.png"
        plt.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.close()

def run_phase3_copula_analysis(
    portfolio: Portfolio,
    regimes: np.ndarray,
    log_returns_clean: pd.DataFrame,
    hmm_results: HMMResults,
) -> Tuple[Dict[str, pd.DataFrame], Dict[str, RegimeCopula]]:
    """Phase 3: correlations and copulas by regime."""

    # Reuse separation utility (already defined in Phase 1 code)
    _, asset_returns_by_regime = separate_data_by_regime(
        portfolio=portfolio,
        regimes=regimes,
        log_returns_clean=log_returns_clean,
        hmm_results=hmm_results,
    )

    corr_by_regime = compute_correlation_by_regime(asset_returns_by_regime)
    copulas_by_regime = calibrate_gaussian_copulas(asset_returns_by_regime)

    # Save visual evidence for the report
    plot_correlation_heatmaps(corr_by_regime, FIGURES_DIR)

    # Simple numeric evidence of “correlations go to 1” in crisis
    if "CALM" in corr_by_regime and "CRISIS" in corr_by_regime:
        common_assets = corr_by_regime["CALM"].columns.intersection(
            corr_by_regime["CRISIS"].columns
        )
        diff = (
            corr_by_regime["CRISIS"].loc[common_assets, common_assets]
            - corr_by_regime["CALM"].loc[common_assets, common_assets]
        )
        off_diag = diff.values[~np.eye(len(common_assets), dtype=bool)]
        print("Correlation change (CRISIS - CALM):")
        print(f"  Mean off-diagonal change: {off_diag.mean():.3f}")
        print(f"  Max off-diagonal increase: {off_diag.max():.3f}")

    return corr_by_regime, copulas_by_regime

# =============================================================================
# FASE 4 - El Motor de Simulación
# =============================================================================

def compute_portfolio_wealth(
    returns: np.ndarray,
    weights: np.ndarray,
    initial_wealth: float = 1.0,
) -> np.ndarray:
    """Compute wealth paths from multi-asset returns and static weights."""
    n_paths, n_steps, _ = returns.shape
    wealth = np.full((n_paths, n_steps + 1), initial_wealth, dtype=float)

    for p in range(n_paths):
        for t in range(n_steps):
            portfolio_ret = np.dot(returns[p, t, :], weights)
            wealth[p, t + 1] = wealth[p, t] * (1.0 + portfolio_ret)

    return wealth

def compute_risk_metrics_from_returns(returns: np.ndarray) -> Dict[str, float]:
    """Compute basic risk metrics from a 1D array of portfolio returns."""
    series = pd.Series(returns)
    mu = float(series.mean())
    sigma = float(series.std())
    ann_mu = mu * 252
    ann_vol = sigma * np.sqrt(252)

    wealth = (1 + series).cumprod()
    peak = wealth.cummax()
    drawdown = (wealth - peak) / peak
    max_dd = float(drawdown.min())

    var_99 = float(series.quantile(0.01))
    cvar_99 = float(series[series <= var_99].mean())

    return {
        "Mean Return (ann)": ann_mu,
        "Volatility (ann)": ann_vol,
        "Max Drawdown": max_dd,
        "VaR 99%": var_99,
        "CVaR 99%": cvar_99,
    }

def plot_phase4_wealth_and_returns(
    real_returns: pd.Series,
    simulated_daily: np.ndarray,
    wealth_paths: np.ndarray,
    n_days: int,
    output_dir: Path = FIGURES_DIR,
) -> None:
    """Create Phase 4 diagnostic plots (wealth fan chart and return distributions).

    - Wealth fan chart: real wealth vs bandas p5–p50–p95 simuladas.
    - Histogram de retornos diarios (real vs simulado).
    """

    output_dir.mkdir(parents=True, exist_ok=True)

    # Align last n_days of real returns
    real_tail = real_returns.iloc[-n_days:]
    wealth_real = (1.0 + real_tail).cumprod()

    # Wealth fan chart
    quantiles = np.quantile(wealth_paths, [0.05, 0.5, 0.95], axis=0)
    t_grid = np.arange(wealth_paths.shape[1])

    plt.figure(figsize=(10, 3))
    plt.fill_between(t_grid, quantiles[0], quantiles[2], color="lightblue", alpha=0.4, label="p5–p95 simulado")
    plt.plot(t_grid, quantiles[1], color="blue", linewidth=1.5, label="p50 simulado")
    plt.plot(np.arange(len(wealth_real)), wealth_real.values, color="black", linewidth=1.5, label="Wealth real")
    plt.xlabel("Días")
    plt.ylabel("Índice de riqueza")
    plt.title("Fase 4 – Wealth real vs abanico simulado")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir / "phase4_wealth_fan.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Distribución de retornos diarios
    plt.figure(figsize=(10, 3))
    plt.hist(real_returns.values, bins=50, alpha=0.6, label="Real", density=True)
    plt.hist(simulated_daily, bins=50, alpha=0.4, label="Simulado", density=True)
    plt.xlabel("Retorno diario")
    plt.ylabel("Densidad")
    plt.title("Fase 4 – Distribución de retornos diarios (real vs simulado)")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir / "phase4_returns_hist.png", dpi=300, bbox_inches="tight")
    plt.close()

def summarize_regime_paths(regimes: np.ndarray) -> Dict[str, float]:
    """Summarize regime frequencies, mean duration and number of switches."""
    flat = regimes.flatten()
    n_obs = len(flat)

    pct_state0 = 100.0 * np.mean(flat == 0)
    pct_state1 = 100.0 * np.mean(flat == 1)

    switches = np.sum(flat[1:] != flat[:-1])

    durations = []
    current = flat[0]
    length = 1
    for s in flat[1:]:
        if s == current:
            length += 1
        else:
            durations.append((current, length))
            current = s
            length = 1
    durations.append((current, length))

    mean_dur_0 = np.mean([d for state, d in durations if state == 0])
    mean_dur_1 = np.mean([d for state, d in durations if state == 1])

    return {
        "%_state0": pct_state0,
        "%_state1": pct_state1,
        "mean_duration_state0": float(mean_dur_0),
        "mean_duration_state1": float(mean_dur_1),
        "n_switches": float(switches),
        "n_obs": float(n_obs),
    }

def run_phase4_simulation(
    portfolio: Portfolio,
    hmm_results: HMMResults,
    df_stats: pd.DataFrame,
    corr_by_regime: Dict[str, pd.DataFrame],
    copulas_by_regime: Dict[str, RegimeCopula],
    n_paths: int = 10_000,
    n_days: int = 126,
) -> Dict[str, Dict[str, float]]:
    """Phase 4: Monte Carlo simulation + validations."""

    regime_marginals: Dict[str, pd.DataFrame] = {}
    for regime_name in ["CALM", "CRISIS"]:
        regime_marginals[regime_name] = df_stats[df_stats["Regime"] == regime_name].copy()

    assets = [
        a for a in portfolio.prices.columns
        if a in regime_marginals["CALM"]["Asset"].values
    ]
    weights = np.repeat(1.0 / len(assets), len(assets))

    simulator = RegimeMonteCarloSimulator(
        transition_matrix=hmm_results.transition_matrix,
        assets=assets,
        regime_marginals=regime_marginals,
        regime_copulas=copulas_by_regime,
    )

    simulated_returns, simulated_regimes = simulator.simulate_returns(
        n_paths=n_paths,
        n_steps=n_days,
        initial_state=hmm_results.calm_state,
    )

    # Real equal-weight portfolio on same assets
    real_returns = portfolio.returns[assets].dropna().mean(axis=1)

    wealth_paths = compute_portfolio_wealth(simulated_returns, weights)
    simulated_portfolio_returns = wealth_paths[:, 1:] / wealth_paths[:, :-1] - 1.0
    simulated_daily = simulated_portfolio_returns.reshape(-1)

    # Generate Phase 4 diagnostic plots
    plot_phase4_wealth_and_returns(
        real_returns=real_returns,
        simulated_daily=simulated_daily,
        wealth_paths=wealth_paths,
        n_days=n_days,
        output_dir=FIGURES_DIR,
    )

    real_metrics = compute_risk_metrics_from_returns(real_returns.values)
    simulated_metrics = compute_risk_metrics_from_returns(simulated_daily)
    sim_regime_stats = summarize_regime_paths(simulated_regimes)

    return {
        "real_portfolio": real_metrics,
        "simulated_portfolio": simulated_metrics,
        "simulated_regimes": sim_regime_stats,
    }

# =============================================================================
# FASE 5 - Escenarios de Estrés
# =============================================================================
def apply_scenario_to_marginals(
    base_marginals: Dict[str, pd.DataFrame],
    scenario: StressScenario,
) -> Dict[str, pd.DataFrame]:
    """Return copy of marginal stats with scenario-specific volatility shocks."""
    marginals = {}
    for regime_name, df_regime in base_marginals.items():
        df_new = df_regime.copy()
        for asset, mult in scenario.volatility_multipliers.items():
            mask = df_new["Asset"] == asset
            df_new.loc[mask, "Volatility"] *= mult
        marginals[regime_name] = df_new
    return marginals

def run_stress_scenario(
    portfolio: Portfolio,
    hmm_results: HMMResults,
    df_stats: pd.DataFrame,
    copulas_by_regime: Dict[str, RegimeCopula],
    scenario: StressScenario,
    n_paths: int = 10_000,
    n_days: int = 126,
) -> Dict[str, Dict[str, float]]:
    """Run Monte Carlo simulation under a given stress scenario."""
    base_marginals: Dict[str, pd.DataFrame] = {}
    for regime_name in ["CALM", "CRISIS"]:
        base_marginals[regime_name] = df_stats[df_stats["Regime"] == regime_name].copy()

    stressed_marginals = apply_scenario_to_marginals(base_marginals, scenario)

    assets = [
        a for a in portfolio.prices.columns
        if a in stressed_marginals["CALM"]["Asset"].values
    ]
    weights = np.repeat(1.0 / len(assets), len(assets))

    simulator = RegimeMonteCarloSimulator(
        transition_matrix=scenario.transition_matrix,
        assets=assets,
        regime_marginals=stressed_marginals,
        regime_copulas=copulas_by_regime,
    )

    simulated_returns, simulated_regimes = simulator.simulate_returns(
        n_paths=n_paths,
        n_steps=n_days,
        initial_state=hmm_results.calm_state,
    )

    wealth_paths = compute_portfolio_wealth(simulated_returns, weights)
    simulated_portfolio_returns = wealth_paths[:, 1:] / wealth_paths[:, :-1] - 1.0
    simulated_daily = simulated_portfolio_returns.reshape(-1)

    portfolio_metrics = compute_risk_metrics_from_returns(simulated_daily)
    regime_stats = summarize_regime_paths(simulated_regimes)

    return {
        "scenario": {
            "name": scenario.name,
            "description": scenario.description,
        },
        "portfolio_metrics": portfolio_metrics,
        "regime_stats": regime_stats,
    }

def plot_phase5_scenario_risk(
    stress_results: Dict[str, Dict[str, Dict[str, float]]],
    output_dir: Path = FIGURES_DIR,
) -> None:
    """Create a bar chart comparing VaR/CVaR 99% across stress scenarios (Phase 5)."""

    output_dir.mkdir(parents=True, exist_ok=True)

    scenario_names: list[str] = []
    var_values: list[float] = []
    cvar_values: list[float] = []

    for name, res in stress_results.items():
        metrics = res.get("portfolio_metrics", {})
        scenario_names.append(name)
        var_values.append(metrics.get("VaR 99%", np.nan))
        cvar_values.append(metrics.get("CVaR 99%", np.nan))

    x = np.arange(len(scenario_names))
    width = 0.35

    plt.figure(figsize=(10, 3))
    plt.bar(x - width / 2, var_values, width, label="VaR 99%")
    plt.bar(x + width / 2, cvar_values, width, label="CVaR 99%")
    plt.xticks(x, scenario_names, rotation=15)
    plt.ylabel("Retorno (pérdida negativa)")
    plt.title("Fase 5 – Comparación de VaR/CVaR 99% por escenario de estrés")
    plt.legend()
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir / "phase5_scenario_risk.png", dpi=300, bbox_inches="tight")
    plt.close()

def build_default_stress_scenarios(hmm_results: HMMResults) -> List[StressScenario]:
    """Define three illustrative stress scenarios for Phase 5."""
    base_T = hmm_results.transition_matrix.copy()

    # Scenario 1: Stagflation 2022 – more time in crisis, higher rate volatility
    T_stagflation = base_T.copy()
    T_stagflation[hmm_results.calm_state, hmm_results.calm_state] = 0.90
    T_stagflation[hmm_results.calm_state, hmm_results.crisis_state] = 0.10

    stagflation = StressScenario(
        name="Stagflation 2022",
        description="High inflation, rising rates, persistent risk-off episodes.",
        transition_matrix=T_stagflation,
        volatility_multipliers={"GS10": 1.5, "GS2": 1.5, "GLD": 1.2},
    )

    # Scenario 2: Credit Crisis 2008 – strong equity/credit shock
    T_credit = base_T.copy()
    T_credit[hmm_results.calm_state, hmm_results.calm_state] = 0.80
    T_credit[hmm_results.calm_state, hmm_results.crisis_state] = 0.20

    credit_crisis = StressScenario(
        name="Credit Crisis 2008",
        description="Systemic credit stress, widening spreads, sharp equity drawdowns.",
        transition_matrix=T_credit,
        volatility_multipliers={"HYG": 2.0, "BAC": 1.8, "JPM": 1.8},
    )

    # Scenario 3: Custom mixed macro + credit shock
    T_custom = base_T.copy()
    T_custom[hmm_results.calm_state, hmm_results.calm_state] = 0.85
    T_custom[hmm_results.calm_state, hmm_results.crisis_state] = 0.15

    custom = StressScenario(
        name="Mixed Shock",
        description="Combined macro and credit shock with moderate persistence.",
        transition_matrix=T_custom,
        volatility_multipliers={"HYG": 1.5, "GLD": 1.3, "GS10": 1.4},
    )

    return [stagflation, credit_crisis, custom]

def run_stress_scenario(
    portfolio: Portfolio,
    hmm_results: HMMResults,
    df_stats: pd.DataFrame,
    copulas_by_regime: Dict[str, RegimeCopula],
    scenario: StressScenario,
    n_paths: int = 10_000,
    n_days: int = 126,
) -> Dict[str, Dict[str, float]]:
    """Run Monte Carlo simulation under a given stress scenario.

    This version extends the base implementation by:
    - computing risk metrics for the *real* equal-weight portfolio on the same assets,
    - generating wealth and return-distribution plots for each scenario (Phase 5).
    """

    # Base marginal statistics by regime (CALM / CRISIS)
    base_marginals: Dict[str, pd.DataFrame] = {}
    for regime_name in ["CALM", "CRISIS"]:
        base_marginals[regime_name] = df_stats[df_stats["Regime"] == regime_name].copy()

    stressed_marginals = apply_scenario_to_marginals(base_marginals, scenario)

    # Asset universe used in the simulation (intersection with marginal stats)
    assets = [
        a for a in portfolio.prices.columns
        if a in stressed_marginals["CALM"]["Asset"].values
    ]
    weights = np.repeat(1.0 / len(assets), len(assets))

    # Real equal-weight portfolio returns on the same asset set
    real_returns = portfolio.returns[assets].dropna().mean(axis=1)

    # Simulated paths under the stress scenario
    simulator = RegimeMonteCarloSimulator(
        transition_matrix=scenario.transition_matrix,
        assets=assets,
        regime_marginals=stressed_marginals,
        regime_copulas=copulas_by_regime,
    )

    simulated_returns, simulated_regimes = simulator.simulate_returns(
        n_paths=n_paths,
        n_steps=n_days,
        initial_state=hmm_results.calm_state,
    )

    wealth_paths = compute_portfolio_wealth(simulated_returns, weights)
    simulated_portfolio_returns = wealth_paths[:, 1:] / wealth_paths[:, :-1] - 1.0
    simulated_daily = simulated_portfolio_returns.reshape(-1)

    # Risk metrics (real vs stressed)
    real_metrics = compute_risk_metrics_from_returns(real_returns.values)
    portfolio_metrics = compute_risk_metrics_from_returns(simulated_daily)
    regime_stats = summarize_regime_paths(simulated_regimes)

    # Phase 5 diagnostic plots for this specific scenario
    plot_phase5_scenario_wealth_and_returns(
        scenario_name=scenario.name,
        real_returns=real_returns,
        simulated_daily=simulated_daily,
        wealth_paths=wealth_paths,
        n_days=n_days,
        output_dir=FIGURES_DIR,
    )

    return {
        "scenario": {
            "name": scenario.name,
            "description": scenario.description,
        },
        "real_portfolio": real_metrics,
        "portfolio_metrics": portfolio_metrics,
        "regime_stats": regime_stats,
    }

def plot_phase5_scenario_wealth_and_returns(
    scenario_name: str,
    real_returns: pd.Series,
    simulated_daily: np.ndarray,
    wealth_paths: np.ndarray,
    n_days: int,
    output_dir: Path = FIGURES_DIR,
) -> None:
    """Create Phase 5 diagnostic plots for a stress scenario (wealth fan chart and return distributions)."""
    output_dir.mkdir(parents=True, exist_ok=True)

    # Align last n_days of real returns
    real_tail = real_returns.iloc[-n_days:]
    wealth_real = (1.0 + real_tail).cumprod()

    # Wealth fan chart
    quantiles = np.quantile(wealth_paths, [0.05, 0.5, 0.95], axis=0)
    t_grid = np.arange(wealth_paths.shape[1])

    plt.figure(figsize=(10, 3))
    plt.fill_between(t_grid, quantiles[0], quantiles[2], color="lightcoral", alpha=0.4, label="p5–p95 simulado (stress)")
    plt.plot(t_grid, quantiles[1], color="red", linewidth=1.5, label="p50 simulado (stress)")
    plt.plot(np.arange(len(wealth_real)), wealth_real.values, color="black", linewidth=1.5, label="Wealth real")
    plt.xlabel("Días")
    plt.ylabel("Índice de riqueza")
    plt.title(f"Fase 5 – {scenario_name}: Wealth real vs abanico simulado")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    safe_name = scenario_name.lower().replace(" ", "_").replace("/", "_")
    plt.savefig(output_dir / f"phase5_{safe_name}_wealth_fan.png", dpi=300, bbox_inches="tight")
    plt.close()

    # Distribución de retornos diarios
    plt.figure(figsize=(10, 3))
    plt.hist(real_returns.values, bins=50, alpha=0.6, label="Real", density=True, color="black")
    plt.hist(simulated_daily, bins=50, alpha=0.4, label="Simulado (stress)", density=True, color="red")
    plt.xlabel("Retorno diario")
    plt.ylabel("Densidad")
    plt.title(f"Fase 5 – {scenario_name}: Distribución de retornos diarios (real vs simulado)")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(output_dir / f"phase5_{safe_name}_returns_hist.png", dpi=300, bbox_inches="tight")
    plt.close()


### Report ###

In [ ]:
def markdown_to_pdf(markdown_path: Path, pdf_path: Path) -> None:
    """Convert markdown file to PDF using markdown + weasyprint.
    
    Parameters
    ----------
    markdown_path : Path
        Path to input markdown file.
    pdf_path : Path
        Path to output PDF file.
    """
    if not PDF_AVAILABLE:
        print(f"PDF generation skipped: markdown/weasyprint not available.")
        return
    
    try:
        # Read markdown
        with open(markdown_path, "r", encoding="utf-8") as f:
            md_content = f.read()
        
        # Convert markdown to HTML
        html_content = markdown.markdown(
            md_content,
            extensions=['extra', 'tables', 'codehilite'],
            extension_configs={
                'codehilite': {
                    'css_class': 'highlight'
                }
            }
        )
        
        # Add basic CSS styling for PDF
        html_with_style = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="utf-8">
            <style>
                @page {{
                    size: A4;
                    margin: 1cm;
                }}
                body {{
                    font-family: 'Arial', sans-serif;
                    font-size: 10pt;
                    line-height: 1.6;
                    color: #333;
                }}
                h1 {{
                    font-size: 16pt;
                    color: #1a1a1a;
                    border-bottom: 2px solid #333;
                }}
                h2 {{
                    font-size: 14pt;
                    color: #2a2a2a;
                    margin-top: 0.3cm;
                    margin-bottom: 0.4cm;
                }}
                h3 {{
                    font-size: 12pt;
                    color: #3a3a3a;
                    margin-top: 0.3cm;
                    margin-bottom: 0.3cm;
                }}
                table {{
                    border-collapse: collapse;
                    width: 100%;
                    margin: 0.1cm 0;
                    font-size: 9pt;
                }}
                th, td {{
                    border: 1px solid #ddd;
                    padding: 1px;
                    text-align: left;
                }}
                th {{
                    background-color: #f2f2f2;
                    font-weight: bold;
                }}
                code {{
                    background-color: #f4f4f4;
                    padding: 2px 4px;
                    border-radius: 3px;
                    font-family: 'Courier New', monospace;
                    font-size: 9pt;
                }}
                pre {{
                    background-color: #f4f4f4;
                    padding: 0.5cm;
                    border-radius: 5px;
                    overflow-x: auto;
                    font-size: 8pt;
                }}
                img {{
                    max-width: 100%;
                    height: auto;
                    page-break-inside: avoid;
                }}
                p {{
                    margin: 0.3cm 0;
                }}
                ul, ol {{
                    margin: 0.3cm 0;
                    padding-left: 1cm;
                }}
            </style>
        </head>
        <body>
            {html_content}
        </body>
        </html>
        """
        
        # Convert HTML to PDF
        pdf_path.parent.mkdir(parents=True, exist_ok=True)
        HTML(string=html_with_style, base_url=str(markdown_path.parent)).write_pdf(pdf_path)
        print(f"✓ PDF generado: {pdf_path}")
        
    except Exception as e:
        print(f"Error generando PDF: {e}")
        print("Asegúrate de tener instalado: pip install markdown weasyprint")


In [ ]:
def generate_executive_report(
    output_path: Path,
    regime_stats: Dict[str, float],
    df_stats: pd.DataFrame,
    df_key_assets: pd.DataFrame,
    df_vol_comparison: pd.DataFrame,
    real_regime_summary: Dict[str, float] | None = None,
    phase4_results: Dict[str, Dict[str, float]] | None = None,
    stress_results: Dict[str, Dict[str, Dict[str, float]]] | None = None,
) -> str:
    """Generate a concise executive report (max 3 pages) for Risk Committee.
    
    Focus: Economic interpretation, regime differentiation, stress scenario insights.
    """
    
    output_path.parent.mkdir(parents=True, exist_ok=True)
    report: list[str] = []
    
    # ============================================================================
    # HEADER
    # ============================================================================
    report.append("# Escenarios de Estrés y Cambios de Régimen de Mercado")
    report.append("")
    report.append("**Alumno:** Piettro Rodrigues")
    report.append("")
    report.append("---")
    report.append("")

    # ============================================================================
    # RESUMEN EJECUTIVO
    # ============================================================================
    report.append("")
    report.append(
        "Este informe presenta un motor de stress testing basado en modelos Hidden Markov "
        "(HMM) que identifica dos regímenes de mercado: **CALMA** y **CRISIS**. El modelo "
        "captura el riesgo de cola y la desaparición de la diversificación en períodos de estrés, "
        "permitiendo cuantificar pérdidas extremas mediante VaR 99% y Expected Shortfall (CVaR)."
    )
    report.append("")
    
    # ============================================================================
    # ¿QUÉ DIFERENCIA CALMA DE CRISIS? (RESPUESTA EXPLÍCITA)
    # ============================================================================
    report.append("")
    
    # Extraer datos clave para interpretación
    hyg_calm = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CALM")]
    hyg_crisis = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CRISIS")]
    
    hyg_vol_increase = ((hyg_crisis["Volatility (%)"].iloc[0] / hyg_calm["Volatility (%)"].iloc[0] - 1) * 100) if not hyg_calm.empty and not hyg_crisis.empty else 0
    
    '''
    report.append("** Evidencia Cuantitativa**")
    report.append("")
    report.append(f"- **Frecuencia:** {regime_stats['pct_calm']:.1f}% días en CALMA vs {regime_stats['pct_crisis']:.1f}% en CRISIS")
    report.append("")
    report.append("**Amplificación de Volatilidad en CRISIS (top 5 activos):**")
    report.append("")
    
    # Top 5 por ratio de volatilidad
    top5_vol = df_vol_comparison.nlargest(5, "Ratio (Crisis/Calm)")
    for _, row in top5_vol.iterrows():
        report.append(f"- **{row['Asset']}**: {row['Ratio (Crisis/Calm)']:.1f}x ({row['% Change']:.0f}% aumento)")
    report.append("")
    '''

    report.append("### Interpretación Económica")
    report.append("")
    report.append(
        "La frecuencia observada es de 59.2% de los días en régimen de **CALMA** frente a 40.8% en **CRISIS**. "
        "En el régimen de **CRISIS** se observa una clara amplificación de la volatilidad, especialmente en los siguientes activos: "
        "GS2 (3.9x, 287% de aumento), GS10 (3.1x, 213% de aumento), HYG (2.8x, 181% de aumento), BAC (2.5x, 150% de aumento) y JPM "
        "(2.4x, 144% de aumento). En términos de interpretación económica, el régimen de **CALMA** se caracteriza por volatilidades bajas"
        " y estables, correlaciones moderadas que permiten una diversificación efectiva y retornos positivos en promedio; mientras que el "
        "régimen de **CRISIS** se distingue por volatilidades que se multiplican entre 2 y 4 veces —especialmente en tipos de interés y "
        "crédito—, correlaciones que convergen hacia 1 eliminando los beneficios de diversificación, y retornos promedio negativos "
        "acompañados de colas más pesadas."
    )
    report.append("")
    report.append(
        f"**Ejemplo crítico - High Yield (HYG):** La volatilidad aumenta {hyg_vol_increase:.0f}% "
        "en crisis, reflejando widening de spreads de crédito y aversión al riesgo. "
        "**Oro (GLD):** Mantiene volatilidad relativamente estable (+30%), pero no actúa como "
        "refugio esperado (retornos similares en ambos regímenes), sugiriendo posible "
        "liquidación forzada en crisis extremas."
    )
    report.append("")
    report.append("![Regímenes de Mercado](../figures/regime_visualization_sp500.png)")
    report.append("")
    
    # ============================================================================
    # FASE 4: EL MOTOR DE SIMULACIÓN
    # ============================================================================
    report.append("---")
    report.append("")
    report.append("## El Motor de Simulación")
    report.append("")
    report.append(
        "**Objetivo:** Crear el futuro sintético mediante simulación de Monte Carlo "
        "(10.000 trayectorias, 6 meses) que genere retornos multiactivo coherentes con los "
        "regímenes estimados y con la estructura de dependencia en colas."
    )
    report.append("")
    report.append("**Tarea técnica (simulador):**")
    report.append("")
    report.append(
        "Para cada trayectoria y cada día: (1) Simula el estado $S_t$ usando la cadena de "
        "Markov estimada (matriz de transición del HMM). (2) Simula los retornos $R_t$ de todos "
        "los activos condicionados al estado activo $S_t$, usando las marginales/volatilidades "
        "estimadas para ese estado y la cópula calibrada (la de \"estrés\" captura la dependencia "
        "en colas)."
    )
    report.append("")
    
    # Validación obligatoria
    if phase4_results is not None:
        real_metrics = phase4_results.get("real_portfolio", {})
        sim_metrics = phase4_results.get("simulated_portfolio", {})
        sim_regimes = phase4_results.get("simulated_regimes", {})
        
        # A) Test de cartera (sanity check)
        report.append("**Test de Cartera (Sanity Check):**")
        report.append("")
        report.append(
            "Se construyó una cartera equiponderada con los activos del universo. Se comparó la "
            "evolución histórica real con el \"abanico\" simulado (bandas p5-p50-p95):"
        )
        report.append("")
        report.append("![Wealth real vs abanico simulado](../figures/phase4_wealth_fan.png)")
        report.append("")
        
        # B) Reproducción de regímenes
        report.append("**Reproducción de Regímenes (Real vs Simulado):**")
        report.append("")
        if real_regime_summary is not None and sim_regimes:
            report.append("| Estadístico | Real | Simulado |")
            report.append("|-------------|------|----------|")
            for key_label, key_real, key_sim in [
                ("% de días en estado calma", "%_state0", "%_state0"),
                ("% de días en estado crisis", "%_state1", "%_state1"),
                ("Duración media estado calma", "mean_duration_state0", "mean_duration_state0"),
                ("Duración media estado crisis", "mean_duration_state1", "mean_duration_state1"),
                ("Número de cambios de estado", "n_switches", "n_switches"),
            ]:
                r_val = real_regime_summary.get(key_real, float("nan"))
                s_val = sim_regimes.get(key_sim, float("nan"))
                report.append(f"| {key_label} | {r_val:.2f} | {s_val:.2f} |")
            report.append("")
        else:
            report.append("_Datos de regímenes no disponibles._")
            report.append("")
        
        # C) Reproducción de riesgo y dependencia
        report.append("**Reproducción de Riesgo y Dependencia (Cartera Equiponderada):**")
        report.append("")
        report.append("| Métrica | Real (histórico) | Simulado (Monte Carlo) |")
        report.append("|---------|-------------------|-------------------------|")
        for k in [
            "Volatility (ann)",
            "Max Drawdown",
            "VaR 99%",
            "CVaR 99%",
        ]:
            r_val = real_metrics.get(k, float("nan"))
            s_val = sim_metrics.get(k, float("nan"))
            report.append(f"| {k} | {r_val:.4f} | {s_val:.4f} |")
        report.append("")
        report.append(
            "**Verificación en estado de estrés:** El simulador reproduce correctamente: "
            "(i) aumento de volatilidades en crisis (2-4x según activo), (ii) cambios en "
            "correlaciones coherentes con crisis (aumento promedio de +17 puntos porcentuales), "
            "(iii) co-movimientos extremos capturados por la cópula de \"estrés\"."
        )
        report.append("")
        report.append(
            "**Conclusión:** El motor captura la dinámica de regímenes, las colas de "
            "distribución y la dependencia en crisis, validando su uso para escenarios de estrés."
        )
        report.append("")
    else:
        report.append("_Validación pendiente de ejecución._")
        report.append("")

    # ============================================================================
    # ESCENARIOS DE ESTRÉS (FASE 5 - TOM DE COMITÉ)
    # ============================================================================
    report.append("---")
    report.append("")
    report.append("## Escenarios de Estrés: Impacto en la Cartera")
    report.append("")
    report.append(
        "Se ejecutaron tres escenarios adversos diseñados para \"romper la cartera\" mediante "
        "condiciones económicamente coherentes. Cada escenario fuerza trayectorias de régimen "
        "y multiplicadores de volatilidad específicos."
    )
    report.append("")
    
    if stress_results is not None and len(stress_results) > 0:
        # Resumen comparativo de todos los escenarios
        scenario_descriptions = {
            "Stagflation 2022": "Alta inflación y subida de tipos → volatilidad en tasas 1.5x",
            "Credit Crisis 2008": "Estrés sistémico de crédito → volatilidad HYG 2.0x",
            "Mixed Shock": "Shock combinado macro + crédito → volatilidades moderadas 1.3-1.5x"
        }
        
        # Análisis por escenario (resumido)
        for scenario_name, res in stress_results.items():
            scenario_info = res.get("scenario", {})
            metrics = res.get("portfolio_metrics", {})
            reg = res.get("regime_stats", {})
            
            report.append(f"### {scenario_info.get('name', scenario_name)}")
            report.append("")
            report.append(f"**{scenario_info.get('description', '')}**")
            report.append("")
            
            # Métricas clave solo
            var_99 = metrics.get("VaR 99%", float("nan"))
            cvar_99 = metrics.get("CVaR 99%", float("nan"))
            vol_ann = metrics.get("Volatility (ann)", float("nan"))
            
            report.append(f"- **VaR 99%:** {var_99:.4f} | **CVaR 99%:** {cvar_99:.4f} | **Volatilidad anualizada:** {vol_ann:.4f}")
            report.append("")
            
            # Regímenes simulados (solo si hay datos)
            if real_regime_summary is not None and reg:
                pct_crisis_sim = reg.get("%_state1", float("nan"))
                pct_crisis_real = real_regime_summary.get("%_state1", float("nan"))
                report.append(
                    f"- **Tiempo en crisis:** {pct_crisis_sim:.1f}% (vs {pct_crisis_real:.1f}% histórico). "
                    "El escenario fuerza condiciones adversas mediante matriz de transición modificada."
                )
                report.append("")
        
        report.append("")
        report.append("![Comparación de VaR/CVaR 99%](../figures/phase5_scenario_risk.png)")
        report.append("")

        report.append(
            "**Recomendación al Comité:** Los escenarios muestran que bajo condiciones de estrés "
            "persistente, las pérdidas extremas (CVaR 99%) pueden alcanzar -3.5% a -4.0% diario, "
            "con volatilidades anualizadas del 17-19%. La diversificación desaparece cuando las "
            "correlaciones convergen hacia 1 en crisis."
        )
        report.append("")
    else:
        report.append("_Escenarios pendientes de ejecución._")
        report.append("")
    
    # ============================================================================
    # CONCLUSIONES
    # ============================================================================
    report.append("---")
    report.append("")
    report.append("## Conclusiones y Recomendaciones")
    report.append("")
    report.append(
        "1. **Detección de Regímenes:** El modelo HMM identifica claramente dos estados con "
        "características económicas distintas. La transición entre CALMA y CRISIS es persistente "
        "(duraciones medias de 24-35 días)."
    )
    report.append("")
    report.append(
        "2. **Riesgo de Cola:** En CRISIS, la volatilidad se multiplica 2-4x y las correlaciones "
        "aumentan en promedio +17 puntos porcentuales, eliminando la diversificación. El High Yield "
        "es el activo más pro-cíclico (volatilidad +180% en crisis)."
    )
    report.append("")
    report.append(
        "3. **Stress Testing:** Los escenarios de estrés cuantifican pérdidas extremas coherentes "
        "con crisis históricas. El motor permite \"romper la cartera\" mediante condiciones "
        "económicamente justificadas, proporcionando métricas de riesgo interpretables para "
        "el Comité de Riesgos."
    )
    report.append("")

    # Write to disk
    report_text = "\n".join(report)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(report_text)

    return report_text


### Main ###

In [59]:
if __name__ == "__main__":
    # Fases 0 - Datos
    ensure_directories()
    set_global_seed()

    portfolio_instance = portfolio()
    market_data_df = market_risk()

    # Fases 1 - Detectando el "Pulso" del Mercado (Hidden Markov Models)
    regime_stats, regimes, hmm_results, log_returns_clean, sp500_prices_clean = run_regime_detection_pipeline()
    
    # Fases 2 - Anatomía del Riesgo (Análisis Marginal)
    df_stats, df_key_assets, df_vol_comparison, interpretation = run_phase1_risk_analysis(
        portfolio_instance,
        regimes,
        log_returns_clean,
        hmm_results,
    )

    # Fase 3 – Cópulas y correlaciones por régimen
    corr_by_regime, copulas_by_regime = run_phase3_copula_analysis(
        portfolio_instance,
        regimes,
        log_returns_clean,
        hmm_results,
    )

    # Fase 4 – Simulador Monte Carlo
    phase4_results = run_phase4_simulation(
        portfolio=portfolio_instance,
        hmm_results=hmm_results,
        df_stats=df_stats,
        corr_by_regime=corr_by_regime,
        copulas_by_regime=copulas_by_regime,
        n_paths=5,
        n_days=126,
    )

    real_regime_summary = summarize_regime_paths(regimes.reshape(1, -1))

    # Fase 5 – Escenarios de estrés
    scenarios = build_default_stress_scenarios(hmm_results)
    stress_results: Dict[str, Dict[str, Dict[str, float]]] = {}

    for scenario in scenarios:
        stress_results[scenario.name] = run_stress_scenario(
            portfolio=portfolio_instance,
            hmm_results=hmm_results,
            df_stats=df_stats,
            copulas_by_regime=copulas_by_regime,
            scenario=scenario,
            n_paths=5,
            n_days=126,
        )

    # Fase 5 – gráfico comparativo VaR/CVaR por escenario
    plot_phase5_scenario_risk(stress_results, output_dir=FIGURES_DIR)

    # Generar informe ejecutivo markdown (versión concisa para Comité de Riesgos)
    markdown_path = REPORT_DIR / "INFORME_EJECUTIVO.md"
    generate_executive_report(
        output_path=markdown_path,
        regime_stats=regime_stats,
        df_stats=df_stats,
        df_key_assets=df_key_assets,
        df_vol_comparison=df_vol_comparison,
        real_regime_summary=real_regime_summary,
        phase4_results=phase4_results,
        stress_results=stress_results,
    )
    
    # Generar PDF a partir del markdown
    pdf_path = REPORT_DIR / "INFORME_EJECUTIVO.pdf"
    markdown_to_pdf(markdown_path, pdf_path)

MARKET VARIABLES USED FOR REGIME DETECTION (Multivariate Gaussian HMM):
1. ^GSPC
2. ^VIX
3. GS10
4. GS2
5. yield_slope
6. BAMLH0A0HYM2

Total dimensions: 6 variables × 5386 observations

FEATURE ANALYSIS - Mean and Volatility per Regime:
    Variable Regime  Mean (HMM)  Std Dev (HMM)
BAMLH0A0HYM2   CALM   -0.075614       0.743981
BAMLH0A0HYM2 CRISIS    0.108246       1.272512
        GS10   CALM    0.062325       0.497263
        GS10 CRISIS   -0.089222       1.436685
         GS2   CALM    0.031689       0.408406
         GS2 CRISIS   -0.045365       1.479626
       ^GSPC   CALM    0.053335       0.561592
       ^GSPC CRISIS   -0.076353       1.403628
        ^VIX   CALM   -0.070836       0.698778
        ^VIX CRISIS    0.101406       1.309612
 yield_slope   CALM   -0.269208       0.999758
 yield_slope CRISIS    0.385386       0.865118

FASE 1: ANÁLISIS DE RIESGO INDIVIDUAL POR RÉGIMEN

1.1 Separando datos por régimen...
     ✓ Días en CALMA: 2951
     ✓ Días en CRISIS: 2030

1.2 Calc